In [ ]:
import os

DATASET_PATH = "/content/final_dataset"

if not os.path.isdir(DATASET_PATH) or len(os.listdir(DATASET_PATH)) == 0:
    print("Dataset belum tersedia, menyalin dari Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive')
    !cp -r /content/drive/MyDrive/final_dataset /content/
else:
    print("Dataset sudah tersedia dan valid.")

Dataset belum tersedia, menyalin dari Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from PIL import Image
import os


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device digunakan:", device)


Device digunakan: cuda


In [ ]:
BASE_DIR = "/content/final_dataset"

TRAIN_DIR = os.path.join(BASE_DIR, "training")
VAL_DIR   = os.path.join(BASE_DIR, "validation")
TEST_DIR  = os.path.join(BASE_DIR, "testing/busuk_segar")


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    )
])


In [ ]:
!ls /content/final_dataset


testing  training  validation


In [ ]:
train_data = datasets.ImageFolder(TRAIN_DIR, transform=transform)
val_data   = datasets.ImageFolder(VAL_DIR, transform=transform)

train_loader = DataLoader(
    train_data,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_data,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print("Class:", train_data.classes)


Class: ['busuk', 'segar']


In [ ]:
import torch
import sys

if not torch.cuda.is_available():
    print("CUDA tidak tersedia. Menginstall PyTorch versi CUDA...")
    !pip uninstall -y torch torchvision torchaudio
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
else:
    print("CUDA sudah tersedia, tidak perlu install ulang.")


CUDA sudah tersedia, tidak perlu install ulang.


In [ ]:
model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 1),
    nn.Sigmoid()
)

model = model.to(device)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 136MB/s]


In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


In [ ]:
from PIL import Image
import os

def clean_dataset(root_dir):
    removed = 0
    for root, _, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                path = os.path.join(root, file)
                try:
                    img = Image.open(path)
                    img.verify()
                except Exception:
                    os.remove(path)
                    removed += 1
    print(f"Total file rusak dihapus: {removed}")

clean_dataset("/content/final_dataset")


Total file rusak dihapus: 0


In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device, non_blocking=True)
        labels = labels.float().unsqueeze(1).to(device, non_blocking=True)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {running_loss:.4f}")


Epoch [1/10] Loss: 232.1597
Epoch [2/10] Loss: 163.4731
Epoch [3/10] Loss: 156.0388
Epoch [4/10] Loss: 146.8524
Epoch [5/10] Loss: 139.1749
Epoch [6/10] Loss: 137.4727
Epoch [7/10] Loss: 131.8631
Epoch [8/10] Loss: 134.1847
Epoch [9/10] Loss: 137.1208
Epoch [10/10] Loss: 131.5361


In [ ]:
def kebusukan_dan_estimasi(prob):
    if prob < 0.25:
        return "aman", 7
    elif prob < 0.5:
        return "rendah", 3
    elif prob < 0.75:
        return "sedang", 1
    else:
        return "parah", 0


In [ ]:
import os
from PIL import Image
import torch

# Path ke folder testing
TEST_DIR = "/content/final_dataset/testing/sample"

model.eval()

# Ekstensi file gambar yang diterima
image_extensions = (".jpg", ".jpeg", ".png")

for filename in os.listdir(TEST_DIR):
    if filename.lower().endswith(image_extensions):
        file_path = os.path.join(TEST_DIR, filename)

        # Load dan preprocessing gambar
        img = Image.open(file_path).convert("RGB")
        img_tensor = transform(img).unsqueeze(0).to(device)

        # Inference
        with torch.no_grad():
            prob_busuk = model(img_tensor).item()

        # Interpretasi hasil
        level, hari = kebusukan_dan_estimasi(prob_busuk)

        # Output
        print("===================================")
        print(f"Nama file            : {filename}")
        print(f"Probabilitas busuk   : {prob_busuk:.2f}")
        print(f"Tingkat kebusukan    : {level}")

        if hari > 0:
            print(f"Layak dikonsumsi hingga : {hari} hari ke depan")
        else:
            print("Makanan ini TIDAK disarankan untuk dikonsumsi")


Nama file            : apel-h3.jpeg
Probabilitas busuk   : 0.94
Tingkat kebusukan    : parah
Makanan ini TIDAK disarankan untuk dikonsumsi
Nama file            : timun-h2.jpeg
Probabilitas busuk   : 0.89
Tingkat kebusukan    : parah
Makanan ini TIDAK disarankan untuk dikonsumsi
Nama file            : timun-h3.jpeg
Probabilitas busuk   : 0.18
Tingkat kebusukan    : aman
Layak dikonsumsi hingga : 7 hari ke depan
Nama file            : apel-h2.jpeg
Probabilitas busuk   : 0.15
Tingkat kebusukan    : aman
Layak dikonsumsi hingga : 7 hari ke depan
Nama file            : timun-h1.jpeg
Probabilitas busuk   : 0.92
Tingkat kebusukan    : parah
Makanan ini TIDAK disarankan untuk dikonsumsi
Nama file            : apel-h6.jpeg
Probabilitas busuk   : 0.04
Tingkat kebusukan    : aman
Layak dikonsumsi hingga : 7 hari ke depan
Nama file            : apel-h4.jpeg
Probabilitas busuk   : 0.97
Tingkat kebusukan    : parah
Makanan ini TIDAK disarankan untuk dikonsumsi
Nama file            : apel-h1.jpeg
Pro

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
from PIL import Image
import torch

model.eval()

for filename in uploaded.keys():
    img = Image.open(filename).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        prob_busuk = model(img_tensor).item()

    level, hari = kebusukan_dan_estimasi(prob_busuk)

    print("===================================")
    print(f"Nama file       : {filename}")
    print(f"Probabilitas busuk : {prob_busuk:.2f}")
    print(f"Tingkat kebusukan  : {level}")

    if hari > 0:
        print(f"Makanan ini layak dikonsumsi hingga : {hari} hari ke depan")
    else:
        print("Makanan ini TIDAK disarankan untuk dikonsumsi")
